In [1]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.global_state import GlobalState, liquidate_against_omnipool
from hydradx.model.amm.agents import Agent
from hydradx.model.amm.liquidations import CDP
from hydradx.model import run



In [5]:
from hydradx.model.amm.global_state import liquidate_against_omnipool_and_settle_otc
from hydradx.model.amm.otc import OTC

# same seed, same parameters = same simulation result
# random.seed(42)

prices = {'DOT': 6, 'HDX': 0.02, 'USDT': 1, 'WETH': 2500, 'iBTC': 45000}

assets = {
    'DOT': {'usd price': prices['DOT'], 'weight': 0.40},
    'HDX': {'usd price': prices['HDX'], 'weight': 0.10},
    'USDT': {'usd price': prices['USDT'], 'weight': 0.30},
    'WETH': {'usd price': prices['WETH'], 'weight': 0.10},
    'iBTC': {'usd price': prices['iBTC'], 'weight': 0.10}
}

lrna_price_usd = 35
initial_omnipool_tvl = 20000000
liquidity = {}
lrna = {}

for tkn, info in assets.items():
    liquidity[tkn] = initial_omnipool_tvl * info['weight'] / info['usd price']
    lrna[tkn] = initial_omnipool_tvl * info['weight'] / lrna_price_usd
    
cdps = [
    CDP('USDT', 'DOT', 100, 20, True),
    CDP('USDT', 'HDX', 100, 10000, False),
    CDP('USDT', 'WETH', 25000000, 10200, True),
]

otcs = [
    OTC('DOT', 'USDT', 100, 6.001, True)
]

initial_state = GlobalState(
    pools={
        'omnipool': OmnipoolState(
            tokens={
                tkn: {'liquidity': liquidity[tkn], 'LRNA': lrna[tkn]} for tkn in assets
            },
            preferred_stablecoin='USDT',
        )
    },
    agents = {'Treasury': Agent(holdings={tkn: 0 for tkn in assets})},
    # cdps=cdps,
    otcs=otcs,
    archive_all=True,
    evolve_function=liquidate_against_omnipool_and_settle_otc('omnipool', 'Treasury'),
)

# print(f'Running with {3} time steps')
events = run.run(initial_state, time_steps=2)

Starting simulation...


KeyboardInterrupt: 

In [3]:
from pprint import pprint

pprint(cdps)
pprint(otcs)
for event in events:
    pprint(event.cdps)
    pprint(event.otcs)
    pprint(event.agents['Treasury'].holdings)

[CDP(USDT, DOT, 100, 20, True),
 CDP(USDT, HDX, 100, 10000, False),
 CDP(USDT, WETH, 25000000, 10200, True)]
[OTC(DOT, USDT, 100, 6.001, True)]
[]
[OTC(DOT, USDT, 4.7779083251953125, 6.001, True)]
{'DOT': 2.2194996063262806e-08,
 'HDX': 0,
 'USD': 0,
 'USDT': 0,
 'WETH': 0,
 'iBTC': 0}
[]
[OTC(DOT, USDT, 4.7779083251953125, 6.001, True)]
{'DOT': 2.2194996063262806e-08,
 'HDX': 0,
 'USD': 0,
 'USDT': 0,
 'WETH': 0,
 'iBTC': 0}


In [4]:
omnipool = events[-1].pools['omnipool']
pprint(omnipool.price(omnipool, 'DOT', 'USDT'))

6.0020001638686935
